### Prediction - Unet with Resnet 34 model 

In [10]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
import os
import matplotlib.pyplot as plt

In [11]:
wd=1e-2

In [12]:
def acc_camvid(input, target):
    target = target.squeeze(1)
    return (input.argmax(dim=1) == target).float().mean()

In [13]:
path_img = Path('/home/ec2-user/SageMaker/data/footprint/9831-train-images/input/')
path_lbl = Path('/home/ec2-user/SageMaker/data/footprint/9831-train-images/gt-binary/')

img_names = get_image_files(path_img)
lbl_names = get_image_files(path_lbl)

get_y_fn = lambda x: path_lbl/f'{x.stem}_gt{x.suffix}'


img_size = 512
bs = 2
codes = array(['None', 'Building'])

src = (SegmentationItemList.from_folder(path_img)
       .split_by_rand_pct()
       .label_from_func(get_y_fn, classes=codes))

data = (src.transform(get_transforms(), size=img_size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(imagenet_stats))

In [14]:
data.classes

array(['None', 'Building'], dtype='<U8')

In [15]:
learn = unet_learner(data, models.resnet50, metrics=acc_camvid, wd=wd)
#learn.load('/home/ec2-user/SageMaker/resnet/weights/best-unet-resnet-entire-best-weights~9894-accuracy-70-epoch')
learn.load('/home/ec2-user/SageMaker/lp02-team-l-model-Research/src/models/footprint/resnet-50/weights/entire-run-4-bestmodel_epoch_is_2_accuracy_is_0.9885603189468384')

Learner(data=ImageDataBunch;

Train: LabelList (7865 items)
x: SegmentationItemList
Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512)
y: SegmentationLabelList
ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512)
Path: /home/ec2-user/SageMaker/data/footprint/9831-train-images/input;

Valid: LabelList (1966 items)
x: SegmentationItemList
Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512)
y: SegmentationLabelList
ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512)
Path: /home/ec2-user/SageMaker/data/footprint/9831-train-images/input;

Test: None, model=DynamicUnet(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-

In [19]:
# test_image_path = '/home/ec2-user/SageMaker/data/footprint/9831-train-images/input/'
# predict_save_image_path = '/home/ec2-user/SageMaker/data/footprint/9831-images-rn50-predictions'

test_image_path = '/home/ec2-user/SageMaker/notebooks/bfp-related-nb/test_rotate'
predict_save_image_path = '/home/ec2-user/SageMaker//notebooks/bfp-related-nb/test_rotate_pred'

### Get all the test images from folder

In [20]:
test_images = [os.path.join(test_image_path, img_name) for img_name in os.listdir(test_image_path) if os.path.isfile(os.path.join(test_image_path, img_name))]
test_images

['/home/ec2-user/SageMaker/notebooks/bfp-related-nb/test_rotate/img_127_45_deg.jpg',
 '/home/ec2-user/SageMaker/notebooks/bfp-related-nb/test_rotate/img_127_180_deg.jpg',
 '/home/ec2-user/SageMaker/notebooks/bfp-related-nb/test_rotate/img_127_270_deg.jpg',
 '/home/ec2-user/SageMaker/notebooks/bfp-related-nb/test_rotate/img_127_90_deg.jpg']

### Prediction on Test Images

In [21]:
for idx, test_image in enumerate(test_images, start = 1):
    print('processing ', idx)
    #Predict the Test Image
    img = open_image(test_image)
    prediction = learn.predict(img)
#     plt.imshow(prediction)
    pred = (prediction[1][0,:,:]).numpy()
    _, img_name = os.path.split(test_image)
    pred_img_name = os.path.splitext(img_name)[0] + '_pred.png'
    plt.imsave(os.path.join(predict_save_image_path, pred_img_name), pred)

    #Create a Mask Image based on the Prediction.
#     mask = np.zeros([512,512,3], dtype=np.uint8)
#     mask[pred == 1] = (0,255,0)    #Enable GREEN channel 

    #store the Mask Image
#     _, img_name = os.path.split(test_image)
#     pred_img_name = os.path.splitext(img_name)[0] + '_pred.png'
#     plt.imsave(os.path.join(predict_save_image_path, pred_img_name), mask)
    
print('total predictions done : ', idx)

processing  1
processing  2
processing  3
processing  4
total predictions done :  4


In [ ]:
import os.path

In [ ]:
path.ls('/home/ec2-user/SageMaker/data/footprint/sample-test-prediction-output')

In [ ]:
from matplotlib.pyplot import imshow
from PIL import Image
import cv2 as cv
import os

In [ ]:
img_preds = [os.path.join(predict_save_image_path, img_pred) for img_pred in os.listdir(predict_save_image_path) if os.path.isfile(os.path.join(predict_save_image_path, img_pred))]
img_preds

In [ ]:
# for img_p in img_preds:
# str1 = os.path.join(predict_save_image_path, 'image_62_pred_binary.png')
img = Image.open('/home/ec2-user/SageMaker/data/footprint/clear-roof-80-to-95-bucket-13-images/pred_output/image_62_pred_binary.png')
imshow(np.asarray(img))

In [ ]:
img = cv.imread('/home/ec2-user/SageMaker/resnet/prediction-result/Image_185_pred.png')
cv.imshow('mg',img)

In [ ]:
#*****************Karthik additions**************************

img = open_image(f'/home/ec2-user/SageMaker/data/footprint/clear-roof-80-to-95-bucket-13-images/pred_output/image_62_pred_binary.png')
prediction = learn.predict(img)
pred = (prediction[1][0,:,:]).numpy()
#mask = np.zeros([512,512,3], dtype = uint8)
mask = np.zeros([512,512,3], np.uint8)
mask[pred == 1] = (0,255,0)
plt.imshow(mask)
# plt.imsave('/home/ec2-user/SageMaker/data/temp/'+f'image_{12}.png',mask)

In [ ]:
from itertools import groupby, chain
from PIL import Image
# !pip install opencv-python
# !pip install scikit-image
import cv2
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm
from skimage import io, color
from skimage import img_as_ubyte

In [ ]:
#*****************Karthik additions**************************
mask=cv2.imread(f'/home/ec2-user/SageMaker/data/temp/'+f'image_{12}.png',0)
print(mask.shape)

image=Image.open(f'/home/ec2-user/SageMaker/data/footprint/test-images-input/image_{12}.jpg')
image=np.array(image)
plt.imshow(image)
x=find_centermost_mask(mask,2000)
x= color.gray2rgb(x)
x = img_as_ubyte(x)
src1=image
src1_mask=x
mask_out=cv2.subtract(src1_mask,src1)
mask_out=cv2.subtract(src1_mask,mask_out)
plt.imshow(mask_out)

In [ ]:
#*****************Karthik additions**************************
def find_centermost_mask(input_mask, min_pixels):
    """
    Take in a binary mask and return a mask of the same shape, with only the centermost contiguous portion.
    If a mask contains no contours (objects), then just return it (it's all zeros anyway).
    """

    # Get all the bits aligned in memory
    input_mask = np.ascontiguousarray(input_mask, dtype=np.uint8)
    contours, _ = cv2.findContours(input_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours or len(contours) == 1:
        return input_mask

    moments = [cv2.moments(contour) for contour in contours]
    # (x, y) tuples of centroid locations for contours (that have area > 0)
    centroids = [
        (m['m10'] / m['m00'], m['m01'] / m['m00'])
        if m['m00'] and m['m00'] >= min_pixels else (0, 0)
        for m in moments
    ]

    if not centroids:
        raise ValidationError(f'No identified footprint contours contain more than {min_pixels} pixels.')

    # dimensions come 1-indexed, but array indicies are 0-indexed
    center = [(dim - 1) / 2 for dim in input_mask.shape]

    # Cacluate (Manhattan) distance from center of image
    distances_from_center = [abs(x - center[0]) + abs(y - center[1]) for x, y in centroids]
    centermost_contour = contours[distances_from_center.index(min(distances_from_center))]

    output_mask = np.zeros(input_mask.shape)
    cv2.fillPoly(output_mask, centermost_contour.transpose((1, 0, 2)), 1)

    return output_mask

### My Own test code - no need to use it

In [ ]:
act_img = open_image(os.path.join(test_image_path, 'Image_185.jpg'))
pred_img = open_image(os.path.join(predict_save_image_path, 'Image_185_pred_binary.png'))

act_img.show(fig_size=(5,5))
pred_img.show(fig_size=(5,5))

In [ ]:


plt.figure(figsize=(10,10))

plt.subplot(1,2,1)
plt.imshow(act_img)

plt.subplot(1,2,2)
plt.imshow(pred_img)


### Rough code - for my own purpose

In [ ]:
display('/home/ec2-user/SageMaker/resnet/prediction-result/img-190-pred.png')

In [ ]:
mask = np.zeros([512,512,3], dtype=np.uint8)
mask = np.zeros([512,512,3])
mask[pred == 1] = (0,255,0)
plt.imsave()

In [ ]:
mask.shape

In [ ]:
pred.shape

In [ ]:
rlt = pred[:,:]
type(rlt)

In [ ]:
mask[pred == 1] = (0, 255, 0)

In [ ]:
plt.imshow(mask)

In [ ]:
plt.imsave('/home/ec2-user/SageMaker/resnet/prediction-result/test.png', mask)